# STag Demonstration

The following iPython Jupyter notebook gives a step-by-step demonstration of how to use STag to get the tag probabilities and the predicted class for a spectra.

# Setup

The first step is to read in the beta values for each of the tags as well as an example spectrum (this can be modified to read in an appropriate spectrum of your choice).

**Note:** You will need to change the 'path' directory to the appropriate one for your own system.

In [1]:
import beta_reader
import numpy as np
import os 

path = os.getcwd()
beta = beta_reader.beta_reader(path)
spectra = '%s/DES15C2aty_C2_combined_150917_v03_b00.fits' % path
name = 'DES15C2aty'
z = 0.149

# Pre-processing

In order to use STag, spectra need to be pre-processed appropriately. This involves filtering, de-redshifting, binning, continuum removal, apodisation, and scaling.

All of these steps are handled by the spectra_preprocessing package, which largely uses methods made for the software [DASH](https://github.com/daniel-muthukrishna/astrodash).

In [2]:
import spectra_preprocessing as sp
from astropy.io import fits

#Read in the fits file of the spectra and extract the flux and wavelength
fits_file = spectra
table = fits.open(fits_file)
flux = table[0].data
w0 = table[0].header['CRVAL1']
dw = table[0].header['CDELT1']
p0 = table[0].header['CRPIX1']
nlam = len(flux)
wave = w0+dw*(np.arange(nlam, dtype='d')-p0)
table.close()

full = np.column_stack((wave, flux))

#Initialise for pre-processing
preProcess = sp.PreProcessing(full, 2500, 10000, 1024)

#Do the pre-processing steps
sfWave, sfFlux, minInd, maxInd, sfZ, sfArea = preProcess.two_column_data(z, smooth=6, minWave=2500, maxWave=10000)

#Do scaling                                                                            
flux_pro = sfFlux/sfArea

Pandas module not installed. DASH will use numpy to load spectral files instead. This can be up to 10x slower.


# Cutting the Spectra

Many of the tags use specific wavelength ranges of the spectrum rather than the whole thing and so we create multiple instances of the original spectrum cut at the corresponding wavelengths for each tag. 

In [3]:
class feature_data(object):
    """a class for holding the wavelength and flux for a specific tag."""
    def __init__(self, label):
        self.label = label
        self.wavelength = []
        self.flux = []

cuts = np.genfromtxt('%s/cuts.txt' % path, dtype=int)
# silicon
si_tag = feature_data('Si')
si_tag.wavelength = cuts[0]
si_tag.flux = flux_pro[si_tag.wavelength[0]:si_tag.wavelength[1]]

#helium
he_tag = feature_data('He')
he_tag.wavelength = cuts[1]
he_tag.flux = flux_pro[he_tag.wavelength[0]:he_tag.wavelength[1]]

#calcium
ca_tag = feature_data('Ca')
ca_tag.wavelength = cuts[2]
ca_tag.flux = flux_pro[ca_tag.wavelength[0]:ca_tag.wavelength[1]]

#Helium double peak
dp_tag = feature_data('He double peak')
dp_tag.wavelength = cuts[3]
dp_tag.flux = flux_pro[dp_tag.wavelength[0]:dp_tag.wavelength[1]]

#Iron
fe_tag = feature_data('Fe')
fe_tag.wavelength = cuts[4]
fe_tag.flux = flux_pro[fe_tag.wavelength[0]:fe_tag.wavelength[1]]

#Sulphur
s_tag = feature_data('S')
s_tag.wavelength = cuts[5]
s_tag.flux = flux_pro[s_tag.wavelength[0]:s_tag.wavelength[1]]

# Tagging

With spectra pre-processed and the necessary cuts made, we can now get the tag probabilities of the spectra and add them to an array ready to be given to the trained classifier.

In [4]:
from tagging import log_reg_two

final = np.zeros([1,10])

#Get Hydrogen tag probabilities
H_result = log_reg_two(flux_pro, beta[0])
final[0][0] = H_result

#Get Silicon tag probabilities
Si_result = log_reg_two(si_tag.flux, beta[1])
final[0][1] = Si_result

#Get Helium emission tag probabilities
He_emi_result = log_reg_two(he_tag.flux, beta[2])
final[0][2] = He_emi_result

#Get Helium p cygni tag probabilities
He_cyg_result = log_reg_two(he_tag.flux, beta[3])
final[0][3] = He_cyg_result

#Get Helium absorption tag probabilities
He_abs_result = log_reg_two(he_tag.flux, beta[4])
final[0][4] = He_abs_result

#Get Hydrogen alpha tag probabilities
H_alp_result = log_reg_two(flux_pro, beta[5])
final[0][5] = H_alp_result

#Get Calcium tag probabilities
Ca_result = log_reg_two(ca_tag.flux, beta[6])
final[0][6] = Ca_result

#Get Helium double peak tag probabilities
dp_result = log_reg_two(dp_tag.flux, beta[7])
final[0][7] = dp_result

#Get Fe tag probabilities
fe_result = log_reg_two(fe_tag.flux, beta[8])
final[0][8] = fe_result

#Get S tag probabilities
s_result = log_reg_two(s_tag.flux, beta[9])
final[0][9] = s_result

# Tag Probabilities

One of the key features of STag is that all of the tags have probabilties, which can be accessed on demand.

In [7]:
tag_names = ['H       ','Si      ','He emi  ','He P-cyg','He abs  ','H-alpha ','Ca      ','He  6678','Fe      ','S       ']
for i in range(0,len(tag_names)):
    print("{0:s} {1:5.3f}".format(tag_names[i],final[0][i]))

H        0.000
Si       0.998
He emi   0.000
He P-cyg 0.000
He abs   0.000
H-alpha  0.007
Ca       1.000
He  6678 0.039
Fe       1.000
S        0.000


# Classifying

We can now make our predictions for the class of the supernova by using the trained model. Since we are using softmax, we use 'np.argmax' to select the class with the highest probability, though one can see the probabilities of all the classes by printing 'class_prob'.

The predicted class is given a number, which corresponds to one of the 5 possible classes:

0 = Type Ia

1 = Type II

2 = Type Ib

3 = Type Ic

4 = Type IIb


In [9]:
import keras
v = keras.__version__
from packaging import version
if version.parse(v) < version.parse('2.5.0'):
    print("You may need to update Keras")

#Load in the trained model
model = keras.models.load_model('%s/Classifier Model.h5' % path)

#Make classification prediction
class_prob = model.predict(final)
preds = np.argmax(class_prob, axis=-1)
print("SN %s (with redshift %.3f) predicted class is %d with a %.3f probability " %  (name,z,preds,class_prob[0][preds]))

SN DES15C2aty (with redshift 0.149) predicted class is 0 with a 0.986 probability 


# Closing Remarks

One can use STag by following the steps outlined in this notebook, and with slight modifications one can adapt this code to run on multiple spectra rather than one at a time. 

Note that the classifying model used has only been trained on the 10 tags shown in this notebook, if one wishes to add additional tags then the model will need to be trained again. A more detailed description of how the tags have been made and how the model was built can be found in our paper: LINK GOES HERE.